## 假设总体规模$n = 10000$ 首先有下面两个公式
<b> 1.无限总体的样本量公式 </b> <br>
$$ n = \frac{Z^2 \cdot p \cdot (1 - p)}{E^2}  $$
<del>
<b> 2.有限总体的样本量公式(error) </b> <br>
$$ n = \frac{Z^2 \cdot p \cdot (1 - p)}{\frac{E^2}{N} + \frac{Z^2 \cdot p \cdot (1-p)}{N}} $$
</del>
<b> 3.有限总体调整公式 </b>
$$ n = \frac{n_0}{1 + \frac{n_0 - 1}{N}} $$

对于题目已经给出了$Z$(置信水平对应的Z值)和E(允许的误差范围) 用其计算出 $n$

## 现在考虑来进行优化,如果我们要抽取n个样本量

我们以10为一组 逐组抽取样本,如果发现当样本量为$x$时样本率为$p_x$ 则提前认为这批零件不合格

对于$p_0 = 10\%$的情况,使用蒙特卡洛进行大量多次数据模拟得出的 $p-n$图如下

<img src=""> $~~~$ ← 一个图

以上得到的数据是 当$p_0 = 10\%$时且已有样本量为$n$时 有$98\%$的概率样本次品率会落入的范围上届

则对于上次的抽取 $p_x$ 就可以取这张表上的数据 因为$2\%$在一次实验中的概率很低,且这样做可以在$p_0 \neq 10\%$时快速结束抽样，以减少抽取的样本量,故采取上述策略优化抽取的样本量个数


In [40]:
import numpy as np
from scipy.stats import binomtest
import matplotlib.pyplot as plt

p0 = 0.1
p1 = 0.15
a = 0.05
# 不妨认为第二类错误概率β与α相同
b = 0.10

# 似然函数的边界
ceil, floor = np.log((1 - b) / a), np.log(b / (1 - a))


# 定义似然函数(对数化)
def L(x, n):
    return x * np.log(p1 / p0) + (n - x) * np.log((1 - p1) / (1 - p0))


total = 10000
Z_95 = 1.65
# 使用无限公式计算的
n1 = int(((Z_95 ** 2) * p0 * (1 - p0)) / (a ** 2))
# 有限调整
n2 = int(n1 / (1 + (n1 - 1) / total))

print("无限公式n =", n1, "有限公式n =", n2)


def extract(cnt: int) -> int:
    c0 = 0
    for i in range(cnt):
        sec = np.random.randint(0, data.size)
        if data[sec] == 0:
            c0 += 1
    return c0


def main():
    # 定义单次抽取的步长
    dc = 5

    c0, cnt = 0, 0
    if io:
        print(f"似然边界为{floor} ~ {ceil}")
    while cnt + dc <= n2:
        c0 += extract(dc)
        cnt += dc
        lv = L(c0, cnt)
        if io:
            print(f"{cnt}里有次品{c0},样本次品率{c0 / cnt},似然值为{lv}")
        if lv <= floor:
            if io:
                print(f"抽{cnt},次品{c0},接受这批零件")
            return True,cnt
        elif lv >= ceil:
            if io:
                print(f"抽{cnt},次品{c0},拒绝这批零件")
            return False,cnt
    if cnt < n2:
        c0 += extract(n2 - cnt)
        cnt = n2
    p = c0 / cnt
    res = binomtest(c0, cnt, p0, alternative="less")
    ret = False
    if res.pvalue > a:
        if io:
            print("抽完，但是接受这批零件")
        ret = True,cnt
    else:
        if io:
            print("抽完，但是拒绝这批零件")
    if io:
        print(f"{cnt}里有次品{c0},样本次品率{p},可能性概率为{res.pvalue}")
    return ret,cnt


if __name__ == "__main__":
    pz = 0.10
    pz_start = 0.01
    pz_end = 0.40

    OCx = []
    OCy = []

    ASNx = []
    ASNy = []

    io = False
    pi = pz_start;
    while pi < pz_end:
        data = np.array([0] * int(pi * total) + [1] * int((1 - pi) * total))
        np.random.shuffle(data)

        OCx.append(pi)
        ASNx.append(pi)

        ok = 0
        s = 0

        for i in range(100):
            flag,cnt = main()
            if flag:
                ok += 1
            s += cnt

        OCy.append(ok / 100)
        ASNy.append(s / 100)

        pi += 0.03

    plt.title("OC")
    plt.xlabel("real p0")
    plt.ylabel("accept p1")
    plt.yticks(ticks = np.arange(0, 1 + 0.05, 0.05))
    plt.plot(OCx, OCy)
    plt.axvline(p0, color = 'r', linestyle = "--")
    plt.axvline(p1, color = 'r', linestyle = "--")

    plt.figure()
    plt.title("ASN")
    plt.xlabel("real p0")
    plt.ylabel("average sample size n")
    plt.yticks([v for v in range(0,100,5)])
    plt.axvline(p0, color = 'r', linestyle = "--")
    plt.axvline(p1, color = 'r', linestyle = "--")
    plt.plot(ASNx, ASNy,color = "#B8860B")

    plt.show()


无限公式n = 98 有限公式n = 97
似然边界为-2.251291798606495 ~ 2.8903717578961645
5里有次品2,样本次品率0.4,似然值为0.6394549746964825
10里有次品3,样本次品率0.3,似然值为0.8162864274448522
15里有次品4,样本次品率0.26666666666666666,似然值为0.9931178801932219
20里有次品5,样本次品率0.25,似然值为1.1699493329415918
25里有次品5,样本次品率0.2,似然值为0.8841572637418484
30里有次品5,样本次品率0.16666666666666666,似然值为0.5983651945421053
35里有次品5,样本次品率0.14285714285714285,似然值为0.31257312534236226
40里有次品6,样本次品率0.15,似然值为0.4894045780907317
45里有次品7,样本次品率0.15555555555555556,似然值为0.6662360308391015
50里有次品8,样本次品率0.16,似然值为0.843067483587471
55里有次品8,样本次品率0.14545454545454545,似然值为0.5572754143877279
60里有次品9,样本次品率0.15,似然值为0.7341068671360973
65里有次品11,样本次品率0.16923076923076924,似然值为1.3735618418325797
70里有次品11,样本次品率0.15714285714285714,似然值为1.0877697726328366
75里有次品12,样本次品率0.16,似然值为1.264601225381207
80里有次品12,样本次品率0.15,似然值为0.9788091561814634
85里有次品13,样本次品率0.15294117647058825,似然值为1.1556406089298328
90里有次品13,样本次品率0.14444444444444443,似然值为0.8698485397300892
95里有次品13,样本次品率0.1368421052631579,似然值为0.5840564705303466
100

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

